# Deploying Machine Learning Models on Azure Kubernetes

<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequisites
 - You need a running AKS cluster with kubernetes>1.8 with kubectl configured to use.
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

## Install helm

In [1]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


## Start Seldon-Core

In [2]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd \
    --set rbac.enabled=false \
    --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Sep  8 11:19:41 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     3s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  3s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          3s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-5977c9fdb8-q8sln  0/1    ContainerCreating  0         3s


NOTES:
NOTES: TODO




In [3]:
!kubectl create namespace seldon

namespace "seldon" created


In [5]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set rbac.enabled=false \
        --namespace seldon

NAME:   seldon-core
LAST DEPLOYED: Sat Sep  8 11:30:00 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                                                READY  STATUS             RESTARTS  AGE
seldon-core-seldon-apiserver-76df4bd96-bwssl        0/1    ContainerCreating  0         1s
seldon-core-seldon-cluster-manager-7756b77f5-gp5k7  0/1    ContainerCreating  0         1s
seldon-core-redis-575979b79b-tnlzj                  0/1    ContainerCreating  0         1s

==> v1/Service
NAME                          TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)                        AGE
seldon-core-seldon-apiserver  NodePort   10.0.53.238   <none>       8080:31360/TCP,5000:30643/TCP  1s
seldon-core-redis             ClusterIP  10.0.157.126  <none>       6379/TCP                       1s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-seldon-apiserver        1        1        1           0          1s


## Set up REST and GRPC methods

## Set up REST and gRPC methods

**Ensure you port forward the seldon api-server REST and GRPC ports**:

REST:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

GRPC:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:5000
```

In [6]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

## Normal Operation

### Create Seldon Deployment

In [7]:
!kubectl create -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [13]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-example-classifier-0 replicas:1 replicasAvailable:1]]]

### Get Predictions

In [14]:
from seldon_utils import *
API_GATEWAY_REST="localhost:8002"
API_GATEWAY_GRPC="localhost:8003"

In [15]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"d9ee378d-8b8c-40ef-b781-bbb78aa71bee","token_type":"bearer","expires_in":43199,"scope":"read write"}
{
  "meta": {
    "puid": "ok6v4n6g6vj1pq7dhmi3tdsqp9",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.0746667772329389]
    }
  }
}


In [16]:
grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"d9ee378d-8b8c-40ef-b781-bbb78aa71bee","token_type":"bearer","expires_in":43196,"scope":"read write"}
meta {
  puid: "bjnij3eq7j6cgcb3oc77s1f6q0"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08000330355299602
  }
}



## Update Deployment with Canary

In [17]:
!kubectl apply -f resources/model_with_canary.json -n seldon

seldondeployment "seldon-model" configured


Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [20]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-example-classifier-0 replicas:1 replicasAvailable:1] map[name:test-deployment-canary-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-canary-mean-classifier-0 replicas:1 replicasAvailable:1]]]

In [21]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"d9ee378d-8b8c-40ef-b781-bbb78aa71bee","token_type":"bearer","expires_in":42969,"scope":"read write"}
{
  "meta": {
    "puid": "4p1rn9548s7mh212h9tihmfi69",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08610190141457626]
    }
  }
}


In [22]:
grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"d9ee378d-8b8c-40ef-b781-bbb78aa71bee","token_type":"bearer","expires_in":42965,"scope":"read write"}
meta {
  puid: "j7bof8jbs413i5frk1h5r7baoh"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09535696733409055
  }
}



## Tear Down

In [ ]:
!kubectl delete -f resources/model.json -n seldon

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge